In [1]:
import torch
import numpy as np
import time
import argparse
import torch.nn as nn
import coremltools as ct
from coremltools.models.neural_network import quantization_utils

In [2]:
def create_mlp(D,N,NumLayers) :
    layers = nn.ModuleList()
    for i in range(NumLayers):
        layers.append(nn.Linear(D, N, bias=False))
    return torch.nn.Sequential(*layers)

def init_weights(m):
    if isinstance(m, nn.Linear):
        torch.nn.init.xavier_normal_(m.weight)

In [3]:
def generate_mlmodels(M, N, batch, Layers):
    model = create_mlp(M,N,Layers)
    model.apply(init_weights)
    model.eval()

    with torch.no_grad():
        x = torch.randn(batch,M)
        output = model(x)

    traced_model = torch.jit.trace(model, x)

    apple_model = ct.convert(traced_model, inputs=[ct.TensorType(name="input", shape = (batch,M) )])
    #filename = str(M)+"x"+str(N)+"x"+str(Layers)+"x"+str(batch)+".mlmodel"
    #apple_model.save(filename)
                     
    apple_model_FP16 = quantization_utils.quantize_weights(apple_model, nbits=16)
    filename = str(M)+"x"+str(N)+"x"+str(Layers)+"x"+str(batch)+"_FP16.mlmodel"
    apple_model_FP16.save(filename)
    
    apple_model_INT8LUT = quantization_utils.quantize_weights(apple_model, nbits=8, quantization_mode="kmeans")
    filename = str(M)+"x"+str(N)+"x"+str(Layers)+"x"+str(batch)+"_INT8LUT.mlmodel"
    apple_model_INT8LUT.save(filename)

In [ ]:
for i in range(7):
    M = 256*2**(i)
    N = M
    for l in range(1,5):
        layers = 8*l
        try:
            generate_mlmodels(M,N,1,layers)
        except GenerateModelError:
            print("Model with dimension ", M, "x", N, " with", lyaers, " layers not created") 

Converting PyTorch Frontend ==> MIL Ops:  94%|▉| 15/16 [00:00<00:00, 3468.28 ops
Running MIL frontend_pytorch pipeline: 100%|█| 5/5 [00:00<00:00, 4414.13 passes/
Running MIL default pipeline:   0%|                 | 0/56 [00:00<?, ? passes/s]/Users/yaof/anaconda3/envs/CoreML/lib/python3.10/site-packages/coremltools/converters/mil/mil/passes/defs/preprocess.py:262: UserWarning: Output, '33', of the source model, has been renamed to 'var_33' in the Core ML model.
  warnings.warn(msg.format(var.name, new_name))
Running MIL default pipeline: 100%|██████| 56/56 [00:00<00:00, 3479.77 passes/s]
Running MIL backend_neuralnetwork pipeline: 100%|█| 8/8 [00:00<00:00, 25811.10 p
Translating MIL ==> NeuralNetwork Ops: 100%|██| 24/24 [00:00<00:00, 87.67 ops/s]


Quantizing using linear quantization
Quantizing layer input.3 of type innerProduct
Quantizing layer input.5 of type innerProduct
Quantizing layer input.7 of type innerProduct
Quantizing layer input.9 of type innerProduct
Quantizing layer input.11 of type innerProduct
Quantizing layer input.13 of type innerProduct
Quantizing layer input of type innerProduct
Quantizing layer 33 of type innerProduct
Quantizing using kmeans quantization
Optimizing Neural Network before Quantization:
Finished optimizing network. Quantizing neural network..
Quantizing layer input.3 of type innerProduct


/Users/yaof/anaconda3/envs/CoreML/lib/python3.10/site-packages/coremltools/models/neural_network/quantization_utils.py:390: ConvergenceWarning: Number of distinct clusters (1) found smaller than n_clusters (256). Possibly due to duplicate points in X.
  ).fit(wf)


Quantizing layer input.5 of type innerProduct


/Users/yaof/anaconda3/envs/CoreML/lib/python3.10/site-packages/coremltools/models/neural_network/quantization_utils.py:390: ConvergenceWarning: Number of distinct clusters (1) found smaller than n_clusters (256). Possibly due to duplicate points in X.
  ).fit(wf)


...

Quantizing layer 129 of type innerProduct


/Users/yaof/anaconda3/envs/CoreML/lib/python3.10/site-packages/coremltools/models/neural_network/quantization_utils.py:390: ConvergenceWarning: Number of distinct clusters (1) found smaller than n_clusters (256). Possibly due to duplicate points in X.
  ).fit(wf)
Converting PyTorch Frontend ==> MIL Ops:  94%|▉| 15/16 [00:00<00:00, 797.58 ops/
Running MIL frontend_pytorch pipeline: 100%|█| 5/5 [00:00<00:00, 9562.94 passes/
Running MIL default pipeline:   0%|                 | 0/56 [00:00<?, ? passes/s]/Users/yaof/anaconda3/envs/CoreML/lib/python3.10/site-packages/coremltools/converters/mil/mil/passes/defs/preprocess.py:262: UserWarning: Output, '33', of the source model, has been renamed to 'var_33' in the Core ML model.
  warnings.warn(msg.format(var.name, new_name))
Running MIL default pipeline: 100%|███████| 56/56 [00:00<00:00, 812.83 passes/s]
Running MIL backend_neuralnetwork pipeline: 100%|█| 8/8 [00:00<00:00, 29511.37 p
Translating MIL ==> NeuralNetwork Ops: 100%|██| 24/24 [00:17

Quantizing using linear quantization
Quantizing layer input.3 of type innerProduct
Quantizing layer input.5 of type innerProduct
Quantizing layer input.7 of type innerProduct
Quantizing layer input.9 of type innerProduct
Quantizing layer input.11 of type innerProduct
Quantizing layer input.13 of type innerProduct
Quantizing layer input of type innerProduct
Quantizing layer 33 of type innerProduct
Quantizing using kmeans quantization
Optimizing Neural Network before Quantization:
Finished optimizing network. Quantizing neural network..
Quantizing layer input.3 of type innerProduct


/Users/yaof/anaconda3/envs/CoreML/lib/python3.10/site-packages/coremltools/models/neural_network/quantization_utils.py:390: ConvergenceWarning: Number of distinct clusters (1) found smaller than n_clusters (256). Possibly due to duplicate points in X.
  ).fit(wf)


Quantizing layer input.5 of type innerProduct
